In [ ]:
import numpy as np
import pandas as pd
import os
x_load=[]
y_load=[]
count=1
# Context
for dirname, _, filenames in os.walk('./quick-draw'):
    for filename in filenames:
        filename=os.path.join(dirname, filename)
        x=np.load(filename)
        print(filename)
        x=x[0:40000,:]
        x=x.astype('float32')/255
        x_load.append(x)
        y=[count for _ in range(40000)]
        count+=1
        y=np.array(y).astype('float32')
        y=y.reshape(y.shape[0],1)
        y_load.append(y)

In [ ]:
x_load=np.array(x_load)
print(x_load.shape)
x_load=x_load.reshape(x_load.shape[0]*x_load.shape[1],x_load.shape[2])
y_load=np.array(y_load).astype('float32')
print(y_load.shape)
y_load=y_load.reshape(y_load.shape[0]*y_load.shape[1],y_load.shape[2])

In [ ]:
from keras.layers import Dense, GRU, Flatten, Conv2D, MaxPooling2D, Dropout, Reshape
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import pickle
model=Sequential()
model.add(Conv2D(32,(5,5),input_shape=(28,28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(Reshape((4,256)))
model.add(GRU(64, return_sequences=False, return_state=False))
model.add(Flatten())
model.add(Reshape((64,)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(9,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.utils import np_utils
y_load = np_utils.to_categorical(y_load)
train_x, test_x, train_y, test_y=train_test_split(x_load, y_load, random_state=0, test_size=0.1)
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
x=model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=5, batch_size=64)
print(x)
model.save('./models/my_model2.h5')